<a href="https://colab.research.google.com/github/ADMaple/Amazon_Vine_Analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.2.2'
os.environ['SPARK_VERSION']=spark_version

In [2]:
# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null

# install spark (change the version number if needed)
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz

# unzip the spark file to the current folder
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

Ign:1 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [985 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:14 

In [3]:
# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

In [4]:
# Start a SparkSession
import findspark
findspark.init()

In [5]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-10-22 02:24:08--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  6.02MB/s    in 0.2s    

2022-10-22 02:24:08 (6.02 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [6]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [7]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://adrianmaplebucket.s3.amazonaws.com/amazon_reviews_us_Outdoors_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
amazon_data_df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)

In [8]:
# Show DataFrame
amazon_data_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   18446823|R35T75OLUGHL5C|B000NV6H94|     110804376|Stearns Youth Boa...|        Outdoors|          4|            0|          0|   N|                Y|          Four Stars|          GOOD VALUE| 2015-08-31|
|         US|   13724367|R2BV735O46BN33|B000IN0W3Y|     624096774|Primal Wear Men's...|        Outdoors|          5|    

**Create the vine Data Frame**







In [10]:
vine_df = amazon_data_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R35T75OLUGHL5C|          4|            0|          0|   N|                Y|
|R2BV735O46BN33|          5|            0|          0|   N|                Y|
|R2NBEUGPQQGXP1|          4|            0|          0|   N|                Y|
|R17LLAOJ8ITK0S|          3|            1|          1|   N|                Y|
|R39PEQBT5ISEF4|          1|            0|          0|   N|                Y|
|R3GNM3SU9VHJFT|          4|            1|          1|   N|                Y|
| R2Y81OP0EK467|          5|            0|          0|   N|                Y|
|R2LFGSI6HAYH5F|          5|            1|          1|   N|                Y|
|R297G6ED1IQO7W|          5|            1|          1|   N|                Y|
| RE27RFC6101N6|          5|            0|          0|   N|     

In [14]:
# filter Data Frame for total_votes above or equal to 20
votes_df = amazon_data_df.filter(vine_df.total_votes >= 20)
votes_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   30222858|R2FP3U4NHNFNL2|B00YPISPNC|     468413405|Stanley Classic V...|        Outdoors|          5|           25|         29|   N|                Y|GREAT GROWLER! NO...|This item is awes...| 2015-08-31|
|         US|   35677754|R1UUK1977O38MU|B00T8NEI3A|      32341693|Camping Wood Burn...|        Outdoors|          5|    

In [15]:
# Filter Data Frame for helpful_votes ratio above or equal to 50%
votes_df_2 = votes_df.filter((votes_df.helpful_votes / votes_df.total_votes) >= 0.5)
votes_df_2.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   30222858|R2FP3U4NHNFNL2|B00YPISPNC|     468413405|Stanley Classic V...|        Outdoors|          5|           25|         29|   N|                Y|GREAT GROWLER! NO...|This item is awes...| 2015-08-31|
|         US|   35677754|R1UUK1977O38MU|B00T8NEI3A|      32341693|Camping Wood Burn...|        Outdoors|          5|    

In [16]:
# Create paid vine Data Frame
paid_df = votes_df_2.filter(votes_df_2.vine == 'Y')
paid_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   43335941|R3KPC0NBUDASX3|B00R8KC02Q|     872035750|Thule EnRoute Tri...|        Outdoors|          5|           25|         25|   Y|                N|Love the Thule ba...|THULE:<br /><br /...| 2015-08-24|
|         US|   36470546|R119P2A95GGXX4|B00NOYKVSK|     165748383|Wenzel Temp Contr...|        Outdoors|          5|    

In [17]:
# Create unpaid vine Data Frame
unpaid_df = votes_df_2.filter(votes_df_2.vine == 'N')
unpaid_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   30222858|R2FP3U4NHNFNL2|B00YPISPNC|     468413405|Stanley Classic V...|        Outdoors|          5|           25|         29|   N|                Y|GREAT GROWLER! NO...|This item is awes...| 2015-08-31|
|         US|   35677754|R1UUK1977O38MU|B00T8NEI3A|      32341693|Camping Wood Burn...|        Outdoors|          5|    

**Find the percentage of paid and unpaid 5-star reviews**
---



In [18]:
# paid total number of reviews
total_paid_reviews = paid_df.count()
total_paid_reviews

107

In [19]:
# paid 5-star reviews
paid_five_star_reviews = paid_df.filter(paid_df.star_rating == 5).count()
paid_five_star_reviews

56

In [20]:
# paid 5-star reviews percentage
paid_five_star_percent = (paid_five_star_reviews / total_paid_reviews) * 100
paid_five_star_percent

52.336448598130836

In [21]:
# unpaid total number of reviews
total_unpaid_reviews = unpaid_df.count()
total_unpaid_reviews

39869

In [22]:
# unpaid 5-star reviews
unpaid_five_star_reviews = unpaid_df.filter(unpaid_df.star_rating == 5).count()
unpaid_five_star_reviews

21005

In [23]:
# unpaid 5-star reviews percentage
unpaid_five_star_percent = (unpaid_five_star_reviews / total_unpaid_reviews) * 100
unpaid_five_star_percent

52.68504351751988